In [2]:
%matplotlib inline

import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys

from PIL import Image
from utils.helpers import *

In [4]:
from src.unet_keras import *

In [5]:
# Loaded a set of images
root_dir = "data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(10, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

Loading 10 images
satImage_052.png
Loading 10 images
satImage_052.png


In [6]:
x_train = np.asarray(imgs)
y_train = np.expand_dims(np.asarray(gt_imgs), axis=3)

In [7]:
x_train.shape

(10, 400, 400, 3)

In [8]:
y_train.shape

(10, 400, 400, 1)

In [9]:
IMG_SIZE = 400
NUM_CHANNELS = 3
NUM_FILTER = 16
FILTER_SIZE = 3

BATCH_SIZE = 16
NUM_EPOCHS = 10

In [10]:
model = unet_model(IMG_SIZE, NUM_CHANNELS, NUM_FILTER, FILTER_SIZE)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model = train_model(model, x_train[:5], y_train[:5], x_train[5:8], y_train[5:8], BATCH_SIZE, NUM_EPOCHS)

Train on 5 samples, validate on 3 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
5/5 [==============================] - 21s 4s/sample - loss: 4.5745 - acc: 0.2992 - val_loss: 2.2542 - val_acc: 0.0946
Epoch 2/10
5/5 [==============================] - 7s 1s/sample - loss: 4.3526 - acc: 0.3128 - val_loss: 2.2290 - val_acc: 0.1686
Epoch 3/10
5/5 [==============================] - 7s 1s/sample - loss: 4.1982 - acc: 0.3173 - val_loss: 2.2024 - val_acc: 0.2178
Epoch 4/10
5/5 [==============================] - 7s 1s/sample - loss: 4.0630 - acc: 0.3204 - val_loss: 2.2058 - val_acc: 0.2325
Epoch 5/10
5/5 [==============================] - 7s 1s/sample - loss: 3.9373 - acc: 0.3322 - val_loss: 2.2717 - val_acc: 0.2387
Epoch 6/10
5/5 [==============================] - 7s 1s/sample - loss: 3.8105 - acc: 0.3492 - val_loss: 2.4179 - val_acc: 0.2372
Epoch 7/10
5/5 [==============================] - 8s 2s/sample - loss: 3.6767 - acc: 0.3710 - val_loss: 2.6372 - val_acc: 0.2351
Epoch 

In [12]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_train[8:], y_train[8:])
print('test loss, test acc:', results)


# Evaluate on test data
2/2 [==============================] - 1s 460ms/sample - loss: 2.7249 - acc: 0.2188
test loss, test acc: [2.7249317169189453, 0.21883126]


In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_train[:3])
print('predictions shape:', predictions.shape)


# Generate predictions for 3 samples
